# AIRAS(AI Research Automation System)

<a href="https://colab.research.google.com/github/auto-res/researchgraph/blob/main/notebook/sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- [Repository](https://github.com/auto-res/airas)
- [Documento](https://auto-res.github.io/airas/)

In [ ]:
import os
from google.colab import userdata

# Please register the secret with the name below in google colab.

# https://platform.openai.com/settings/organization/api-keys
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
# https://app.devin.ai/settings/api-keys
os.environ["DEVIN_API_KEY"] = userdata.get("DEVIN_API_KEY")
# https://www.firecrawl.dev/app/api-keys
os.environ["FIRE_CRAWL_API_KEY"] = userdata.get("FIRE_CRAWL_API_KEY")
# https://docs.github.com/ja/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens#fine-grained-personal-access-token-%E3%81%AE%E4%BD%9C%E6%88%90
os.environ["GITHUB_PERSONAL_ACCESS_TOKEN"] = userdata.get("GITHUB_PERSONAL_ACCESS_TOKEN")
os.environ["VERTEX_AI_API_KEY"] = userdata.get("VERTEX_AI_API_KEY")

In [ ]:
!git clone https://github.com/auto-res/airas.git

In [ ]:
%%capture
%cd /content/airas
%pip install -r requirements.txt
%cd ../../

In [ ]:
import glob

import sys
sys.path.append('/content/airas/src')

import logging
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

logging.basicConfig(
    level=logging.INFO,
    format='[%(levelname)s] %(name)s: %(message)s'
)

In [ ]:
# You can change the repository name to anything you like. Example: auto-res2/tanaka-1
# Once the repository is created, you don't need to change it again.
github_repository = "auto-res2/sample"

# You can also rename the branch as you like.
# One branch corresponds to one unit of research.
# If you run the same subgraph more than once on the same branch, a new branch will be created (a new research fork).
# You can continue your research by running subsequent subgraphs on the forked branch.
# You can check the forked branches by changing the repository name in the link below and accessing it.
# https://github.com/auto-res2/test26/network
branch_name = "test"
save_dir = "/content/airas/data"

### Prepare the repository

In [ ]:
from airas.research_preparation_subgraph.research_preparation_subgraph import ResearchPreparationSubgraph

In [ ]:
preparer = ResearchPreparationSubgraph(
    device_type="gpu",
    organization="auto-res2",
)

input = {
    "github_repository": github_repository,
    "branch_name": branch_name,
}
result = preparer.run(input)
print(result)

# Retrieve Papers

In [ ]:
from airas.retrieve_paper_subgraph.retrieve_paper_subgraph import Retriever

In [ ]:
scrape_urls = [
    "https://icml.cc/virtual/2024/papers.html?filter=title",
    "https://iclr.cc/virtual/2024/papers.html?filter=title",
    # "https://nips.cc/virtual/2024/papers.html?filter=title",
    # "https://cvpr.thecvf.com/virtual/2024/papers.html?filter=title",
]
add_paper_num = 1

retriever = Retriever(
    github_repository=github_repository,
    branch_name=branch_name,
    perform_download=False,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
    scrape_urls=scrape_urls,
    add_paper_num=add_paper_num,
)

retriever_input = {
    "queries": ["diffusion model"],
}

result = retriever.run(retriever_input)
print(result)

# Generate New Method

In [ ]:
from airas.generator_subgraph.generator_subgraph import Generator

In [ ]:
generator = Generator(
    github_repository=github_repository,
    branch_name=branch_name,
    llm_name="o1-2024-12-17",
)

result = generator.run()
print(f"result: {result}")

# Generate Experimental Code

In [ ]:
from airas.experimental_plan_subgraph.experimental_plan_subgraph import ExperimentalPlaner

In [ ]:
experimentalplaner = ExperimentalPlaner(
    github_repository=github_repository,
    branch_name=branch_name,
)

result = experimentalplaner.run()
print(f"result: {result}")

# Run Experiments

In [ ]:
from airas.executor_subgraph.executor_subgraph import Executor

In [ ]:
max_code_fix_iteration = 3

executor = Executor(
    github_repository=github_repository,
    branch_name=branch_name,
    save_dir=save_dir,
    max_code_fix_iteration=max_code_fix_iteration,
)

result = executor.run()
print(f"result: {result}")

# Analyze Experimental Results

In [ ]:
from airas.analytic_subgraph.analytic_subgraph import Analyst

In [ ]:
analyst = Analyst(
    github_repository=github_repository, branch_name=branch_name, llm_name="o1-2024-12-17"
)

result = analyst.run()
print(f"result: {result}")

# Write Paper

In [ ]:
from airas.writer_subgraph.writer_subgraph import PaperWriter

In [ ]:
refine_round = 1

paper_writer = PaperWriter(
    github_repository=github_repository,
    branch_name=branch_name,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
    refine_round=refine_round,
)

result = paper_writer.run({})
print(f"result: {result}")

# Summarize Research Results in README.md

In [ ]:
from airas.readme_subgraph.readme_subgraph import ReadmeUploader

In [ ]:
readme_uploader = ReadmeUploader(
    github_repository=github_repository,
    branch_name=branch_name,
)

result = readme_uploader.run()
print(f"result: {result}")

# Export Paper as HTML

In [ ]:
import glob
from airas.html_subgraph.html_subgraph import HtmlConverter

In [ ]:
figures_dir = f"{save_dir}/images"
pdf_files = glob.glob(os.path.join(figures_dir, "*.pdf"))

extra_files = [
    {
        "upload_branch": "gh-pages",
        "upload_dir": "branches/{{ branch_name }}/",
        "local_file_paths": [f"{save_dir}/index.html"],
    },
    {
        "upload_branch": "gh-pages",
        "upload_dir": "branches/{{ branch_name }}/images/",
        "local_file_paths": pdf_files,
    },
]

html_converter = HtmlConverter(
    github_repository=github_repository,
    branch_name=branch_name,
    extra_files=extra_files,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
)

result = html_converter.run()
print(f"result: {result}")

# Export Paper as PDF

In [ ]:
!apt-get update && apt-get install -y \
    texlive-base \
    texlive-latex-recommended \
    texlive-fonts-recommended \
    texlive-latex-extra \
    texlive-science \
    chktex \
    locales && \
    rm -rf /var/lib/apt/lists/*

In [ ]:
from airas.latex_subgraph.latex_subgraph import LatexConverter

In [ ]:
extra_files = [
    {
        "upload_branch": "{{ branch_name }}",
        "upload_dir": ".research/",
        "local_file_paths": [f"{save_dir}/paper.pdf"],
    }
]

latex_converter = LatexConverter(
    github_repository=github_repository,
    branch_name=branch_name,
    extra_files=extra_files,
    llm_name="o3-mini-2025-01-31",
    save_dir=save_dir,
)

result = latex_converter.run({})
print(f"result: {result}")